In [1]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import pandas as pd
import numpy as np
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import re

# Clean and tokenize text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords') # download stopwords corpus
nltk.download('punkt') # download punkt tokenizer


2023-05-10 12:37:18.434516: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Hannah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Hannah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the data
df = pd.read_csv('transformers_dataNew.csv')

# Split the data into train, validation, and test sets
train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")

# Tokenize and encode the text data
train_tokens = tokenizer.batch_encode_plus(
    train_data['content'].tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors='tf'
)

valid_tokens = tokenizer.batch_encode_plus(
    valid_data['content'].tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors='tf'
)

test_tokens = tokenizer.batch_encode_plus(
    test_data['content'].tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors='tf'
)

# Convert the tokenized input to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    train_data['entropy'].values
))

valid_dataset = tf.data.Dataset.from_tensor_slices((
    dict(valid_tokens),
    valid_data['entropy'].values
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    test_data['entropy'].values
))

# Prepare the model input
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

# Load the BERT model
model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # Tensorflow

# Freeze the BERT layers
model.trainable = False

# Retrieve the BERT embeddings
embeddings = model(input_ids, attention_mask=attention_mask)[0]

# Perform pooling (average pooling in this case)
pooled_output = tf.keras.layers.GlobalAveragePooling1D()(embeddings)

# Add a dense layer for regression
output = tf.keras.layers.Dense(1, activation='linear')(pooled_output)

# Build the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mean_squared_error')

# Train the model
model.fit(
    train_dataset.batch(32),
    epochs=5,
    validation_data=valid_dataset.batch(32)
)

# Evaluate the model on the test set
predictions = model.predict(test_dataset.batch(32))
mse = mean_squared_error(test_data['entropy'].values, predictions)
print('Mean Squared Error:', mse)


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

## From https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794, Language model approach


In [2]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline
df = pd.read_csv('Indicator-Desc_Data.csv')

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [ ]:
df

In [3]:
def clean_and_tokenize(text):
    # Remove punctuation
    
    text = str(text).translate(str.maketrans("", "", string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('dutch'))
    tokens = [token for token in tokens if not token in stop_words]
    
    return tokens

In [4]:
df['prep_content'] = df['content'].apply(clean_and_tokenize)

In [ ]:
df

In [28]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # Tensorflow


Some layers from the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
corpus = df['content']
maxlen = 100

# add special tokens
maxqnans = np.int((maxlen-20)/2)
corpus_tokenized = ["[CLS] "+
             " ".join(tokenizer.tokenize(re.sub(r'[^\w\s]+|\n', '', 
             str(txt).lower().strip()))[:maxqnans])+
             " [SEP] " for txt in corpus]

## generate masks
masks = [[1]*len(txt.split(" ")) + [0]*(maxlen - len(
           txt.split(" "))) for txt in corpus_tokenized]
    
## padding
txt2seq = [txt + " [PAD]"*(maxlen-len(txt.split(" "))) if len(txt.split(" ")) != maxlen else txt for txt in corpus_tokenized]
    
## generate idx
idx = []
for seq in txt2seq:
    seq.split(" ")
    idx.append(tokenizer.encode(seq))

/var/folders/24/zyx78_1d5fndpjnbr8x7dd6r0000gn/T/ipykernel_2176/1644811398.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  maxqnans = np.int((maxlen-20)/2)


In [33]:
idx

[[1,
  1,
  15441,
  20722,
  11130,
  8529,
  13644,
  19207,
  0,
  0,
  13940,
  30025,
  0,
  0,
  19883,
  126,
  12341,
  0,
  0,
  11281,
  10669,
  22243,
  9144,
  10871,
  10567,
  20722,
  15401,
  0,
  0,
  10127,
  11281,
  10532,
  12999,
  22674,
  9322,
  16058,
  14707,
  10811,
  30040,
  0,
  0,
  30042,
  0,
  0,
  30034,
  16069,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  2],
 [1,
  1,
  11315,
  22592,
  22445,
  0,
  0,
  30042,
  12377,
  13644,
  13261,
  9719,
  11840,
  0,
  0,
  22795,
  0,
  0,
  11281,
  0,
  0,
  54,
  11281,
  13261,
  17390,
  13644,
  13261,
  9719,
  13117,
  22976,
  14157,
  18853,
  16779,
  22592,
  8067,
  17191,
  15266,
  16058,
  12314,
 

In [32]:
## generate segments
segments = [] 
for seq in txt2seq:
    temp, i = [], 0
    for token in seq.split(" "):
        temp.append(i)
        if token == "[SEP]":
             i += 1
    segments.append(temp)
    
## feature matrix
X = [np.asarray(idx, dtype='int32'), 
    np.asarray(masks, dtype='int32'), 
    np.asarray(segments, dtype='int32')]

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (10000,) + inhomogeneous part.

In [22]:
corpus = df['content']
maxlen = 100

## add special tokens
maxqnans = int((maxlen-20)/2)

corpus_tokenized = ["[CLS] " + " ".join(tokenizer.tokenize(re.sub(r'[^\w\s]+|\n', '', str(txt).lower().strip()))[:maxqnans])+ " [SEP] " for txt in corpus]

## generate masks
masks = [[1]*len(txt.split(" ")) + [0]*(maxlen - len(txt.split(" "))) for txt in corpus_tokenized]
    
## padding
txt2seq = [txt + " [PAD]"*(maxlen-len(txt.split(" "))) if len(txt.split(" ")) != maxlen else txt for txt in corpus_tokenized]
    


In [23]:
txt2seq

['[CLS] medewerkers van een aquarium in span ##je a ##ten gerecht ##en die waren bereid door deelnemers van master ##chef en dat hadden ze beter niet kunnen doen r ##t ##l nieuws [SEP]  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] er wordt wie ##t gerookt in het boek ge ##zo ##en ##d en het personage in het boek heeft één keer seks ook wordt abortus overwogen maar niet gepleegd r ##t ##l nieuws [SEP]  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [25]:
# Generate idx and segments
idx, segments = [], []
for seq in txt2seq:
    temp_idx, temp_segment = tokenizer.tokenize(first=seq)
    idx.append(temp_idx)
    segments.append(temp_segment)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [18]:

# Generate idx and segments
idx, segments = [], []
for seq in txt2seq:
    temp_idx, temp_segment = tokenizer.encode(first=seq)
    idx.append(temp_idx)
    segments.append(temp_segment)


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
## generate segments
segments = [] 
for seq in txt2seq:
    temp, i = [], 0
    for token in seq.split(" "):
        temp.append(i)
        if token == "[SEP]":
             i += 1
    segments.append(temp)## feature matrix
X_train = [np.array(idx, dtype='int32'), 
           np.array(masks, dtype='int32'), 
           np.array(segments, dtype='int32')]

## Nieuwe approach

In [9]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
#model = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # PyTorch
model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # Tensorflow

Some layers from the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
df['prep_content']

0       [medewerkers, aquarium, spanje, aten, gerechte...
1       [wiet, gerookt, boek, gezoend, personage, boek...
2       [andy, team, dragbar, laten, ondanks, schrik, ...
3       [slachtoffers, volgens, vnmensenrechtenchef, v...
4       [alleen, afgelopen, drie, dagen, kwamen, 3000,...
                              ...                        
9995    [elke, aardbewoner, surft, jaarlijks, gemiddel...
9996    [nederlander, team, red, bull, hadden, wind, a...
9997    [„, we, weten, weinig, mogelijke, effecten, pr...
9998                    [rommelt, binnen, cda, telegraaf]
9999                                   [balen, telegraaf]
Name: prep_content, Length: 10000, dtype: object

In [10]:
tokenized_texts = [tokenizer.encode(text, max_length=512, truncation=True) for text in df['prep_content']]


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
from keras.preprocessing.sequence import pad_sequences
max_length = max([len(txt) for txt in tokenized_texts])
padded_texts = pad_sequences(tokenized_texts, maxlen=max_length, dtype="long", value=0, truncating="post")


In [ ]:

# Encode the text
inputs = tokenizer.encode_plus(
    df['content'],
    add_special_tokens=True,
    max_length=128,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='tf'
)

# Feed the encoded inputs into the BERT model to get the output
bert_model = transformers.TFBertModel.from_pretrained('bert-base-uncased')
output = bert_model(inputs)[0]

# Apply average pooling to summarize the output
pooled_output = tf.reduce_mean(output, axis=1)

# Add the final dense layers to predict the final absolute value
x = tf.keras.layers.Dense(64, activation='relu')(pooled_output)
y_out = tf.keras.layers.Dense(1, activation='linear')(x)

# Create the model
model = tf.keras.models.Model(inputs=inputs, outputs=y_out)

# compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])